In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [5]:
df_raw = pd.read_csv('BankChurners.csv')

In [6]:
df_raw.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


First I will make y, our target 1 or zero.  To do this i will first make col attrition flag a category type the try to binary it.

In [25]:
dfy = df_raw[['CLIENTNUM','Attrition_Flag']]
df = pd.get_dummies(dfy['Attrition_Flag'])
df = pd.concat((dfy,df),axis=1)
df = df.drop(["Existing Customer"],axis=1)
y = df['Attrited Customer'].to_numpy()

In [47]:
#===========================================
cols = df_raw.describe().columns
cols = cols.tolist()
dfx = df_raw[cols[1:-2]]
X_num = dfx.to_numpy()
print(X_num.shape)
# now find X_num_gend
df_gender = pd.get_dummies(df_raw['Gender'])
genF = df_gender['F'].to_numpy()
genM = df_gender['M'].to_numpy()

# print(gender.reshape((10127,1)))
X_num_gend = np.append(X_num,genF.reshape((10127,1)),axis=1)
X_num_gend = np.append(X_num_gend,genM.reshape((10127,1)),axis=1)
print(X_num_gend.shape)

(10127, 14)
(10127, 16)


In [43]:
def logreg_num(X,y):
    print('Results for simple Logistic Regression on only numerical columns.')
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)
    logreg = LogisticRegression(max_iter=250)
    logreg.fit(X_train,y_train)
    y_pred = logreg.predict(X_test)
    cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
    print(cnf_matrix)
    tp = cnf_matrix[0,0]
    fp = cnf_matrix[1,0]
    fn = cnf_matrix[0,1]
    tn = cnf_matrix[1,1]
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    print('Precision: ' + str(precision))
    print('Recall: ' + str(recall))
    print('F1 Score: ' + str(2*precision*recall/(precision+recall)))
    print('==========================================================================')

In [44]:
def logreg_num_gend(X,y):
    print('Results for simple Logistic Regression with one categorical column for gender.')
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)
    logreg = LogisticRegression(max_iter=250)
    logreg.fit(X_train,y_train)
    y_pred = logreg.predict(X_test)
    cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
    print(cnf_matrix)
    tp = cnf_matrix[0,0]
    fp = cnf_matrix[1,0]
    fn = cnf_matrix[0,1]
    tn = cnf_matrix[1,1]
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    print('Precision: ' + str(precision))
    print('Recall: ' + str(recall))
    print('F1 Score: ' + str(2*precision*recall/(precision+recall)))
    print('==========================================================================')

In [48]:
print('==========================================================================')
logreg_num_gend(X_num_gend,y)


Results for simple Logistic Regression with one categorical column for gender.
[[2072   76]
 [ 190  194]]
Precision: 0.9160035366931919
Recall: 0.9646182495344506
F1 Score: 0.9396825396825397


In [ ]:
#Results
# ==========================================================================
# Results for simple Logistic Regression on only numerical columns.
# [[2075   73]
#  [ 200  184]]
# Precision: 0.9120879120879121
# Recall: 0.9660148975791434
# F1 Score: 0.9382771874293466
# ==========================================================================

# with one column male = 1, female = 0, we get this
# ==========================================================================
# Results for simple Logistic Regression with one categorical column for gender.
# [[2071   77]
#  [ 196  188]]
# Precision: 0.913542126157918
# Recall: 0.9641527001862198
# F1 Score: 0.9381653454133636
# ==========================================================================

# with one column male = 0, female = 1, we get this, why are they different and this one is better in some ways
# ==========================================================================
# Results for simple Logistic Regression with one categorical column for gender.
# [[2073   75]
#  [ 192  192]]
# Precision: 0.9152317880794701
# Recall: 0.9650837988826816
# F1 Score: 0.9394969408565601
# ==========================================================================

# with column for male and female, we get this, slightly better
# ==========================================================================
# Results for simple Logistic Regression with two categorical columns for gender.
# [[2072   76]
#  [ 190  194]]
# Precision: 0.9160035366931919
# Recall: 0.9646182495344506
# F1 Score: 0.9396825396825397
# ==========================================================================

Do I want to optimize for Recall, then use the beta function for F1.  73 False Negatives, is that too many?

In [56]:
#===================================================================   to test what is going on here on simple data
#===================================================================   Sample Logistic Regression
X = np.array([[14, 2], [9, 13], [1, 18], [5, 12], [11, 6], [19, 2],[7,16],[8,15], [10,10],[5,17]])
y = np.array([1,0,0,0,1,1,0,0,1,0])

In [57]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)

In [58]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

In [59]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

In [60]:
cnf_matrix

array([[1, 0],
       [1, 1]], dtype=int64)

In [61]:
cnf_matrix.shape

(2, 2)

In [62]:
y_pred

array([0, 0, 1])

In [63]:
y_test

array([0, 1, 1])

In [43]:
t = np.array([[1,2,3],[4,5,6]])
t2 = np.array([[7,8]])
t2 = np.transpose(t2)
t

array([[1, 2, 3],
       [4, 5, 6]])

In [44]:
new = np.append(t,t2,axis=1)

In [45]:
new

array([[1, 2, 3, 7],
       [4, 5, 6, 8]])